# Modules

Python allows you to save your functions and classes in another file and then import them into the program you are working on.

A python module most of the time is a simple python file with code inside, e.g. `my_module.py`. You could execute a module with `python my_module.py`.

> Note: module naming convention is **snake_case**

The more common use case is to `import` a module in your code where you need the functionality provided by that module.

There are a number of ways you can import objects you are interested in.

## import *module_name*

In [ ]:
import my_module

value = my_module.some_function()
print(value)

> Note: Using this type of import our `some_function` is inside the namespace of `my_module`.

## from *module_name* import *

Can be used to `import` into the **current namespace**

> Note: **it is bad practice!**

In [ ]:
from my_module import *

value = some_function()
print(value)

If you want to limit what objects are imported using the `import *` approach - you can include `__all__` attribute inside your module.

```python
__all__ = ['some_function']

def some_function() -> str:
    return "Hello module!"

def other_function() -> str:
    return "Not imported with *"
```

`other_function` will not be accessible

In [ ]:
try:
    value = other_function()
except NameError as error:
    print(error)

## from *module_name* import *object*

Allows to `import` only the functionality you actually need.

Using this approach you can also import `other_function`

In [ ]:
from my_module import some_function, other_function

value = some_function()
print(value)
value = other_function()
print(value)

Importing into the current namespace does not prevent from name clashes.

It is usually a better idea to keep the namespace of the module.

## import *module_name* as *local_name*

You may have seen this many times with some popular Python packages (numpy, pands, etc.) to have a shorter name.

In [ ]:
import my_module as mm
value = mm.some_function()

> Note: This saves you from name clashes if you have a local variable, e.g., `animal`, and you need to import a module that is also named `animal`

# Module resolution

Python searches some **system dependent** locations for modules

In [ ]:
%%bash
/usr/bin/python3 -c 'import sys; print(sys.path)'

* `''` - current folder (might be others variations depending on system)
* `/Users/user/Library/Python/3.9/lib/python/site-packages` - version dependent user folder for packages on MacOs. Everything you install with `python -m pip install --user` goes there. To find out the user base of your python installation you can run `python -m site --user-base`.

If you use virtualenv - result is different. It make Python look only inside viartualenv packages.

In [ ]:
import sys
print(sys.path)


## Modifying search path

### Outside of Python

Use the `PYTHONPATH` environment variable to extend the search path to your own locations. `PYTHONPATH` behaves the same as `PATH` variable on your OS.

If a module can not be found, you will get a `ModuleNotFoundError`.

In [ ]:
%%bash
export PYTHONPATH="$(pwd)/example_package:${PYTHONPATH}"
/usr/bin/python3 -c 'import sys; print(sys.path)'

### Inside Python code

Useful for development purposes (no need to change external environment variables) and debugging in Jupyter.

In [ ]:
# Live-reload of imported modules
%load_ext autoreload
%autoreload 2

# Ensure that example_package is importable
import os
import sys
REPO_DIRNAME = os.path.abspath('.')
if REPO_DIRNAME not in sys.path:
    sys.path.append(REPO_DIRNAME)

# You can copy and edit this code to point to where your other Python files are.
# I use current dir because i'm lazy

from example_package.example import SampleClass

# Module execution

When you import a module, it is parsed and executed by interpreter.

In [ ]:
import hello

If you have some code that you want to run when you launch a module using the `python my_module.py` but avoid executing it when you import the module you can check if the top level environment's `__name__` attribute of the module is called `'__main__'` (meaning it is executed directly by interpreter) before executing any code:

```python
def my_function():
    pass

def main():
    my_function()

if __name__ == '__main__':
    main()
```

We can check the difference:

In [ ]:
print(f"This is the '__name__' of currently executing environment: {__name__}")
import hello as hi # we still execute this in between with current implementation
print(f"This is the '__name__' of imported module: {hi.__name__}")

# Python packages

Modules are a great way to organize your code into logical units.

This one-level organization is usually not deep enough for larger projects.

Packages allow you to organize your project hierarchically.

Example package hierarchy:

In [ ]:
!tree cool_package -I '__pycache__'

## `__init__.py` file

The `__init__.py` is used for package-level initialization either when your package is imported or a module within the package is imported.

You write normal python code in that file which is then executed when the package is imported (once).

You can use the `__all__` list inside `__init__.py` to define what should be imported when someone does `from cool_package import *`.

> Note: Often the file is empty. In Python 3.3+ you do not need to have the file if it is empty.

## import packages

You import a package or a nested sub-package just like a module.

In [ ]:
import cool_package.sub_package_1.module_2

cool_package.sub_package_1.module_2.fun2()

To make your life easier you can use `__init__.py`.

For example we can do this in our main package (see subpackes for their details).

```python
from .module_1 import fun1
from .sub_package_1 import fun2
from .sub_package_2 import fun3

__all__ = ['fun1', 'fun2', 'fun3']
```

Now we can do better :)

In [ ]:
import cool_package as cp

cp.fun1()
cp.fun2()
cp.fun3()

In [ ]:
dir(cp)

## Installing and creating redistributable packages

Playing with `pip` and packaging will be left as an excercise :)

Guide on [Installing Packages](https://packaging.python.org/en/latest/tutorials/installing-packages/)

Guide on [Packaging Python Projects](https://packaging.python.org/en/latest/tutorials/packaging-projects/)

### A side note on creating the list of required packages for your application

Usualy when you do the development process you do ad-hoc package installation using pip. When you share your project to coleagues you need to share the list of used packages. You can create the `requirements.txt` manualy, but it may take so time.

Here's a little lifehack - you can use `pip freeze` to get all the packages from current environment (remember, we're using virtualenv per project). In Linux you can redirect this output to a file:

```bash
pip freeze > ./requirements.txt
```

# Unit testing

Python has 3 *main* frameworks for unit testing:

* unittest ([https://docs.python.org/3/library/unittest.html](https://docs.python.org/3/library/unittest.html)): unit testing framework that is part of the standard library.
* doctest ([https://docs.python.org/3/library/doctest.html](https://docs.python.org/3/library/doctest.html)): a test module that allows to write simple unit tests as part of the documentation.
* pytest ([https://docs.pytest.org/en/7.2.x/](https://docs.pytest.org/en/7.2.x/)): alternative 3rd party testing framework. It is compatible to run tests written with the `unittest` package.

## unittest

In `unittest` you create classes that inherit from `unittest.TestCase`. You can use these classes to organize your tests.

Each class defines methods for the tests.
* They must again start with `test_`.

You test your code by calling different `self.assert*` methods (inherited, [see documentation for the entire list](https://docs.python.org/3/library/unittest.html#assert-methods)).

It is also possible to use built in mock library [https://docs.python.org/3/library/unittest.mock.html](https://docs.python.org/3/library/unittest.mock.html)
Example simple unit test:

```python
import unittest # python standard library. No need to install

from testing.package.example import *

class TestExample(unittest.TestCase):
    def test_addition(self):
        result = testable_function(2, 2, sum)
        self.assertEqual(result, 4)
    
    # ... other functions

if __name__ == '__main__':
    unittest.main()
```

To run the tests you need to either execute the test module itself or use unittest module:

```bash
python -m unittest path/to/your_tests.py
```

> Note: for notebook i use a built in magic function

In [ ]:
%run ./testing/tests/example_tests.py

## Code coverage

Generating coverage reports in python is easy.

Two main packages:
* coverage [https://pypi.org/project/coverage/](https://pypi.org/project/coverage/)
* pytest-cov ([https://pypi.org/project/pytest-cov/](https://pypi.org/project/pytest-cov/) a plugin for pytest)

In [ ]:
%%capture
%pip install coverage

In [ ]:
!coverage run -m testing.tests.example_tests

In [ ]:
!coverage report -m